# Elevation Interpolation
## Tzu-Yu Ma
### April 11, 2024

In [71]:
import arcpy
import os
import psycopg2
import random

#### Workspace

In [ ]:
arcpy.env.workspace = r"D:\spring2024\GIS5572\Lab3\Lab3\Lab3.gdb"

#### Connect to PostGIS database

In [73]:
# connect to PostGIS database
conn = psycopg2.connect(
    dbname="gis5572",
    user="postgres",
    password="",
    host="",
    port="5432"
)
cur = conn.cursor()

#### Prepare DEM data

In [19]:
# Execute a query
cur.execute("SELECT * FROM mndem_ponits")

# Fetch the results
results = cur.fetchall()

# Print the results
print(len(results))

# Close the cursor and connection
cur.close()
conn.close()

24778


In [40]:
# copy the data from PostGIS database
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="\\PostgreSQL-34-gis5572(postgres).sde\gis5572.postgres.mndem_ponits",
    Output_Geodatabase=arcpy.env.workspace
)

<Result 'D:\\spring2024\\GIS5572\\Lab3\\Lab3\\Lab3.gdb'>

#### Samples percentage

In [17]:
ele_data = os.path.join(arcpy.env.workspace, 'mndem_ponits')
data = int(arcpy.GetCount_management(ele_data).getOutput(0))

# define the percentage
percentage = 10

percentage_sample = int(data * percentage / 100)

# create a sequence of indices from 1 to the total number of data points
indices = list(range(1, data + 1))

# sample from the indices
sampled_indices = random.sample(indices, percentage_sample)

print("Sampled size:", len(sampled_indices))

Sampled size: 2477


In [18]:
points = []
with arcpy.da.SearchCursor(ele_data, ["SHAPE@XY", "grid_code"]) as cursor:
    for row in cursor:
        points.append(row)

# random selected
random.shuffle(points)

sampled_points = points[:percentage_sample]

# create a new feature class to store sampled points
output_fc = os.path.join(arcpy.env.workspace, "sampled_ele")

if arcpy.Exists(output_fc):
    arcpy.Delete_management(output_fc)

arcpy.CreateFeatureclass_management(out_path=arcpy.env.workspace, out_name="sampled_ele", geometry_type="POINT", spatial_reference=26915) #SR may need to change to 4326 

arcpy.AddField_management(output_fc, "ID", "LONG")
arcpy.AddField_management(output_fc, "grid_code", "LONG")


# insert attribute to sampled_ele 
with arcpy.da.InsertCursor(output_fc, ["SHAPE@XY", "ID", "grid_code"]) as cursor:
    for idx, point in enumerate(sampled_points):
        x, y = point[0]
        point_geometry = arcpy.PointGeometry(arcpy.Point(x, y))
        cursor.insertRow([point_geometry, idx+1, point[1]])

print("New feature class inserted to:", output_fc)


New feature class inserted to: D:\spring2024\GIS5572\Lab3\Lab3\Lab3.gdb\sampled_ele


#### Interpolation algorithms results

In [ ]:
# ORDINARY_KRIGING
with arcpy.EnvManager(scratchWorkspace=arcpy.env.workspace):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="sampled_ele",
        z_field="grid_code",
        kriging_model="Spherical 2268.000000 # # #",
        cell_size= "dem_aggrega",
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(arcpy.env.workspace + "Kriging_ord")

In [ ]:
# UNIVERSAL_KRIGING
with arcpy.EnvManager(scratchWorkspace=arcpy.env.workspace ):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="sampled_ele",
        z_field="grid_code",
        kriging_model="LinearDrift 2268.000000 # # #",
        cell_size="dem_aggrega",
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(arcpy.env.workspace + "Kriging_uni")

In [ ]:
# IDW
with arcpy.EnvManager(scratchWorkspace=arcpy.env.workspace):
    out_raster = arcpy.sa.Idw(
        in_point_features="sampled_ele",
        z_field="grid_code",
        cell_size=r"D:\spring2024\GIS5572\Lab2\Lab2\Lab2.gdb\dem_aggrega",
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster.save(arcpy.env.workspace + "\\Idw")

#### Saves the interpolate results to PostGIS

In [8]:
def raster_pts_SDE(input_raster, output_pt_name):
    arcpy.conversion.RasterToPoint(
        in_raster=input_raster,
        out_point_features=arcpy.env.workspace + "\\" + output_pt_name,
        raster_field="Value")
    
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features=arcpy.env.workspace + "\\" + output_pt_name,
        Output_Geodatabase="\\PostgreSQL-34-gis5572(postgres).sde")
    
    print(f"{input_raster} has been converted to points and stored into SDE as {output_pt_name}.")


In [9]:
# ORDINARY_KRIGING
raster_pts_SDE ('Kriging_ord', 'Kriging_ord_pts')

# UNIVERSAL_KRIGING
raster_pts_SDE ('Kriging_uni', 'Kriging_uni_pts')

# IDW
raster_pts_SDE ('Idw', 'Idw_pts')

Kriging_ord has been converted to points and stored into SDE as Kriging_ord_pts.
Kriging_uni has been converted to points and stored into SDE as Kriging_uni_pts.
Idw has been converted to points and stored into SDE as Idw_pts.


####  Accuracy Assessment - Model 

In [ ]:
# use Exploratory Interpolation to compare Ordinary Kriging, Universal Kriging, and IDW
arcpy.ga.ExploratoryInterpolation(
    in_features="sampled_ele",
    value_field="grid_code",
    out_cv_table="ExploratoryInterpolation",
    out_geostat_layer=None,
    interp_methods="ORDINARY_KRIGING;UNIVERSAL_KRIGING;IDW",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

In [ ]:
# save the table into SDE
arcpy.conversion.TableToGeodatabase(
    Input_Table="ExploratoryInterpolation",
    Output_Geodatabase="\\PostgreSQL-34-gis5572(postgres).sde"
)

#### Accuracy Assessment - Results

In [9]:
def differences(true_dem, interpolated_dem, output):
    # caculate original and interpolated DEMs difference
    difference_raster = arcpy.sa.Minus(true_dem, interpolated_dem)
    
    # get the name of interpolation method as part of the output point layer 
    interpolation = os.path.basename(interpolated_dem)
    
    # define the name of the output point layer
    difference_points = f"diff_pts_{interpolation}"
    
    # convert DEM difference to point layer
    arcpy.RasterToPoint_conversion(difference_raster, difference_points)

    # rename "grid_code" to "difference"
    field_mapping = arcpy.FieldMappings()
    field_mapping.addTable(difference_points)
    for field in field_mapping.fields:
        if field.name == "grid_code":
            field.name = "difference"
    arcpy.management.AlterField(difference_points, "grid_code", "difference")

    print(f"Difference {interpolation} points layer created.")


In [11]:
true_dem = r"D:\spring2024\GIS5572\Lab2\Lab2\Lab2.gdb\dem_aggrega"
output = arcpy.env.workspace

# ORDINARY_KRIGING
interpolated_dem = arcpy.env.workspace + "\\Kriging_ord"
differences(true_dem, interpolated_dem, output)

# UNIVERSAL_KRIGING
interpolated_dem = arcpy.env.workspace + "\\Kriging_uni"
differences(true_dem, interpolated_dem, output)

# IDW
interpolated_dem = arcpy.env.workspace + "\\Idw"
differences(true_dem, interpolated_dem, output)

Difference Kriging_ord points layer created.
Difference Kriging_uni points layer created.
Difference Idw points layer created.


#### store the accuracy assessment to PostGIS database

In [3]:
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="diff_pts_Idw;diff_pts_Kriging_uni;diff_pts_Kriging_ord",
    Output_Geodatabase="\\PostgreSQL-34-gis5572(postgres).sde"
)


<Result 'C:\\Users\\ma000551\\Desktop\\school\\lab3\\lab3\\34.133.74.255.sde'>

#### Adjust data for uploading to AGOL

In [ ]:
# use Using BERNOULLI sampling method to reduce the points
cur.execute("CREATE TABLE sampled_idw_pts_ele AS SELECT * FROM idw_pts TABLESAMPLE BERNOULLI(1);")

# commit changes to the database
conn.commit()

# close the cursor and connectio
cur.close()
conn.close()

In [74]:
# check how many data after using BERNOULLI 
cur.execute("SELECT * FROM sampled_idw_pts_ele")

# fetch the results
results = cur.fetchall()

print(len(results))
# close the cursor and connection
cur.close()
conn.close()

413


In [ ]:
# transfer coordinate to EPSG:4326
cur.execute("""
ALTER TABLE sampled_idw_pts_ele 
ALTER COLUMN shape TYPE geometry(Point, 4326) 
USING ST_Transform(shape, 4326);
""")
# Commit changes to the database
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()